### Problems
1. torchscript из модели detectron2, что можно конвертировать трейсингом (backbone — ok, RPN — failed). Возможно здесь стоит использовать еще scripting. 
2. tensorpack переводит detectron2 модель (не densepose) в статический граф TF, из него можно получить TFLite. Но это тоже вопрос для изучения
3. fasterrcnn/maskrccn из torchvision можно преобразовать в ONNX (буквально со вчера). Но используя backend caffe2 не удалось исполнить эти модели, у них opset ONNX 10 и 11, а в caffe — 9.
4. На backend tf так же не удается запустить модели из ONNX

### 1. Detecron2 -> TorchScript
Needed pytorch>=1.3, detectron2
In next section it is needed to install pytorch-nightly, so install it here

In [1]:
import sys
sys.path.append('./DensePose/')

In [2]:
from detectron2.config import get_cfg
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data import DatasetMapper

from DensePose.densepose import add_densepose_config


import torch
from torch import nn
from torch.nn import functional as F

#### Loading DensePose model

In [4]:
config_file = './DensePose/configs/densepose_rcnn_R_50_FPN_s1x.yaml'
model_path = './DensePose/output/model_final.pth'
opts = ['MODEL.WEIGHTS', model_path]
cfg = get_cfg()
add_densepose_config(cfg)
cfg.merge_from_file(config_file)
cfg.merge_from_list(opts)
cfg.freeze()

model = build_model(cfg)
model = model.eval()

DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR).resume_or_load(
            cfg.MODEL.WEIGHTS, resume=False
)
mapper = DatasetMapper(cfg, is_train=False)

Config './DensePose/configs/densepose_rcnn_R_50_FPN_s1x.yaml' has no VERSION. Assuming it to be compatible with latest v2.


In [5]:
inpput = [mapper({'file_name': './DensePose/2.jpg'})]
images = model.preprocess_image(inpput)
images.tensor.shape

torch.Size([1, 3, 768, 1344])

#### Create backbone wrapper and do tracing --- ok

In [6]:
class backbone_wrapper(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
    def forward(self, x):
        fp = self.backbone(x)
        return tuple(fp.values())
    
backbone_example = images
backbone = model._modules['backbone'].eval()
backbone_wrap = backbone_wrapper(backbone)
traced_backbone = torch.jit.trace(backbone_wrap, backbone_example.tensor.to('cuda:0'))

/root/anaconda2/envs/python3/lib/python3.7/site-packages/torch/jit/__init__.py:1030: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Not within tolerance rtol=1e-05 atol=1e-05 at input[0, 58, 32, 244] (-0.13124431669712067 vs. -0.131231427192688) and 17 other locations (0.00%)
  check_tolerance, _force_outplace, True, _module_class)
/root/anaconda2/envs/python3/lib/python3.7/site-packages/torch/jit/__init__.py:1030: TracerWarning: Output nr 2. of the traced function does not match the corresponding output of the Python function. Detailed error:
Not within tolerance rtol=1e-05 atol=1e-05 at input[0, 180, 83, 91] (0.03434910625219345 vs. 0.03433563560247421) and 16 other locations (0.00%)
  check_tolerance, _force_outplace, True, _module_class)
/root/anaconda2/envs/python3/lib/python3.7/site-packages/torch/jit/__init__.py:1030: TracerWarning: Output nr 3. of the traced function does not match the correspon

#### Create RPN wrapper and do tracing --- failed

In [7]:
from __future__ import division
from typing import Any, List, Sequence, Tuple, Union

class ImageList(object):
    def __init__(self, tensor: torch.Tensor, image_sizes: List[Tuple[int, int]]):
        """
        Arguments:
            tensor (Tensor): of shape (N, H, W) or (N, C_1, ..., C_K, H, W) where K >= 1
            image_sizes (list[tuple[int, int]]): Each tuple is (h, w).
        """
        self.tensor = tensor
        self.image_sizes = image_sizes

    def __len__(self) -> int:
        return len(self.image_sizes)
    
    @staticmethod
    def from_tensors(
        tensors: Sequence[torch.Tensor], size_divisibility: int = 0, pad_value: float = 0.0
    ) -> "ImageList":
        """
        Args:
            tensors: a tuple or list of `torch.Tensors`, each of shape (Hi, Wi) or
                (C_1, ..., C_K, Hi, Wi) where K >= 1. The Tensors will be padded with `pad_value`
                so that they will have the same shape.
            size_divisibility (int): If `size_divisibility > 0`, also adds padding to ensure
                the common height and width is divisible by `size_divisibility`
            pad_value (float): value to pad

        Returns:
            an `ImageList`.
        """
        assert len(tensors) > 0
        assert isinstance(tensors, (tuple, list))
        for t in tensors:
            assert isinstance(t, torch.Tensor), type(t)
            assert t.shape[1:-2] == tensors[0].shape[1:-2], t.shape
        # per dimension maximum (H, W) or (C_1, ..., C_K, H, W) where K >= 1 among all tensors
        max_size = tuple(max(s) for s in zip(*[img.shape for img in tensors]))

        if size_divisibility > 0:
            import math

            stride = size_divisibility
            max_size = list(max_size)  # type: ignore
            max_size[-2] = int(math.ceil(max_size[-2] / stride) * stride)  # type: ignore
            max_size[-1] = int(math.ceil(max_size[-1] / stride) * stride)  # type: ignore
            max_size = tuple(max_size)

        image_sizes = [im.shape[-2:] for im in tensors]

        if len(tensors) == 1:
            # This seems slightly (2%) faster.
            # TODO: check whether it's faster for multiple images as well
            image_size = image_sizes[0]
            padded = F.pad(
                tensors[0],
                [0, max_size[-1] - image_size[1], 0, max_size[-2] - image_size[0]],
                value=pad_value,
            )
            batched_imgs = padded.unsqueeze_(0)
        else:
            batch_shape = (len(tensors),) + max_size
            batched_imgs = tensors[0].new_full(batch_shape, pad_value)
            for img, pad_img in zip(tensors, batched_imgs):
                pad_img[..., : img.shape[-2], : img.shape[-1]].copy_(img)

        return ImageList(batched_imgs.contiguous(), image_sizes)
    
class rpn_wrapper(nn.Module):
    def __init__(self, rpn, size_divisibility):
        super().__init__()
        self.rpn = rpn
        self.size_divisibility = size_divisibility
        self.fp_keys = ['p2', 'p3', 'p4', 'p5', 'p6']
        
    def forward(self, x, fp_values):
        images = ImageList.from_tensors([x], self.size_divisibility)
        features = {k:v for (k,v) in zip(self.fp_keys, fp_values)}
        proposals, losses = self.rpn(images, features, None)
        proposals = proposals[0]._fields['proposal_boxes'].tensor
        return proposals

In [8]:
images = [x["image"].to('cuda:0') for x in inpput]
images = [model.normalizer(x) for x in images]

rpn = model._modules['proposal_generator'].eval()
rpn_wrap = rpn_wrapper(rpn, backbone.size_divisibility).eval()

image = images[0]
fp = backbone(backbone_example.tensor.to('cuda:0'))
fp_values = tuple(fp.values())
rpn_wrap(image, fp_values)

tensor([[ 695.4552,  166.5025,  972.2765,  706.4489],
        [ 265.3167,  125.6099,  600.0546,  722.2170],
        [ 519.1044,   42.9846,  788.3871,  554.9060],
        ...,
        [ 557.5628,  435.4629,  587.3651,  535.2599],
        [1325.5300,  332.0747, 1333.0000,  501.1771],
        [  15.2157,  481.1315,   63.8876,  623.2001]], device='cuda:0')

In [9]:
image = images[0]
fp = backbone(backbone_example.tensor.to('cuda:0'))
fp_values = tuple(fp.values())

example = (image, fp_values, )
traced_script_module = torch.jit.trace(rpn_wrap, example)

/root/anaconda2/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:46: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
/root/anaconda2/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:47: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
/root/anaconda2/envs/python3/lib/python3.7/site-packages/detectron2/modeling/anchor_generator.py:179: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This me

TracingCheckError: Tracing failed sanity checks!
Encountered an exception while running the trace with test inputs.
Exception:
	Dimension out of range (expected to be in range of [-1, 0], but got 1)
	The above operation failed in interpreter, with the following stack trace:
	/root/anaconda2/envs/python3/lib/python3.7/site-packages/detectron2/modeling/proposal_generator/rpn_outputs.py(406): predict_proposals
	/root/anaconda2/envs/python3/lib/python3.7/site-packages/detectron2/modeling/proposal_generator/rpn.py(172): forward
	/root/anaconda2/envs/python3/lib/python3.7/site-packages/torch/nn/modules/module.py(525): _slow_forward
	/root/anaconda2/envs/python3/lib/python3.7/site-packages/torch/nn/modules/module.py(539): __call__
	<ipython-input-7-48f4e0422e9d>(80): forward
	/root/anaconda2/envs/python3/lib/python3.7/site-packages/torch/nn/modules/module.py(525): _slow_forward
	/root/anaconda2/envs/python3/lib/python3.7/site-packages/torch/nn/modules/module.py(539): __call__
	/root/anaconda2

### 2. Tensorpack
```git clone tensorpack```

Follow:
https://github.com/tensorpack/tensorpack/tree/master/examples/FasterRCNN

Creates .pb tf graph

NOTE: coco 2017 dataset is needed

### 3. FastRCNN to ONNX

NOTE: pytorch-nightly (2019oct29) is needed

In [11]:
import torch
import torchvision

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()
x = [torch.rand(3, 300, 400)]
predictions = model(x)

torch.onnx.export(model, (x,), './fasterrcnn.onnx', do_constant_folding=True, opset_version=10)

#### Then try to infer using Caffe2

In [ ]:
import onnx

# Load the ONNX model
model = onnx.load("./fasterrcnn.onnx")

onnx.checker.check_model(model)

onnx.helper.printable_graph(model.graph)

import caffe2.python.onnx.backend as backend
import numpy as np

backend._known_opset_version = 10
rep = backend.prepare(model, device="CUDA:0")
outputs = rep.run(np.random.rand(3, 300, 400).astype(np.float32))
print(outputs[0])

### 4. ONNX -> TF

```pip install onnx-tf```

In [1]:
import onnx

from onnx_tf.backend import prepare

onnx_model = onnx.load("./fasterrcnn.onnx")  # load onnx model
tf_rep = prepare(onnx_model)  # prepare tf representation
tf_rep.export_graph("./fasterrcnn.pb")  # export the model




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/root/anaconda2/envs/python3/lib/python3.7/site-packages/onnx_tf/common/__init__.py:87: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)


/root/anaconda2/envs/python3/lib/python3.7/site-packages/onnx_tf/common/handler_helper.py:37: UserWarning: Unknown op ConstantFill in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/root/anaconda2/envs/python3/lib/python3.7/site-packages/onnx_tf/common/handler_helper.py:37: UserWarning: Unknown op ImageScaler in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))



Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


NotImplementedError: Resize is not implemented.